# Introduction
Use this notebook to create the pickled version of the data for later loading in other notebooks.

# Setup

In [1]:
import scanpy as sc
import pandas as pd
from IPython.display import display 
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle as pkl
from tqdm.notebook import tqdm

import subprocess
import gzip
import sys
import os

In [2]:
mountpoint = '/data/clue/'
prefix_prod = mountpoint + 'prod/'
prefix_pks = mountpoint + 'amo/atac/peaks/'
prefix_eqtl = mountpoint + 'prod/eqtl/'

## Functions

In [4]:
def get_num_headerlines(vcf_path):
    with gzip.open(vcf_path, 'rt') as file:
        i = 0
        for line in file:
            if line[0] == '#':
                i += 1
            else:
                break
    return i

In [5]:
def load_vcf(vcf_full_path):
    # I HATEEEEEEE VCFs
    # the fact that this code needs to be so complicated must mean it is not a very good format
    # everything should be stored in SEPARATE compressed data matrices, separate GT/GP etc.

    vcf = dict() # the values of the dict themselves will also be dictionaries
    vcf['path'] = vcf_full_path # add the path as a dictinoary key and value

    # want to skip a certain number, assuming your VCF does not have a header > 1000 lines
    
    vcf['header_lines_num'] = get_num_headerlines(vcf['path'])
    vcf['header_lines_num'] = vcf['header_lines_num'] - 1 # still want to keep the last line

    # read the file
    vcf['file'] = gzip.open(vcf['path'],'rt')
    for i in range(vcf['header_lines_num']):
        vcf['file'].readline()

    # create a list to build
    vcf['gts'] = list()
    vcf['uids'] = list()
    vcf['info'] = list()

    # add the single-line header
    header = vcf['file'].readline().strip().split('\t')

    # for the rest of them, take the most important information only
    for line in tqdm(vcf['file']):
        line = line.strip().split('\t')

        uid = ":".join([line[0], line[1], line[3], line[4]])
        vcf['uids'].append(uid)

        gts = [i.split(":")[0] for i in line[9:]]
        vcf['gts'].append(gts)

        infoline = [i.split('=') for i in line[7].split(";") if "=" in i]
        vcf['info'].append(dict(zip([i[0] for i in infoline], [i[1] for i in infoline])))

    vcf['info'] = pd.DataFrame(vcf['info'],
                                       index=vcf['uids']).astype(np.float32)
    vcf['info'].index.name='ID'

    vcf['gts'] = pd.DataFrame(vcf['gts'], 
                                     index=vcf['uids'],
                                     columns=header[9:])
    vcf['gts'].index.name='ID'

    vcf['uids'].append(uid)
    # delete the list to reduce memory now
    vcf['file'].close()
    del(vcf['file'])
    return vcf

In [6]:
def make_refalt_geno_dict(name_split):
    return {
        '0|0': name_split[-2] + '/' + name_split[-2],
        '0|1': name_split[-2] + '/' + name_split[-1],
        '1|0': name_split[-2] + '/' + name_split[-1],
        '1|1': name_split[-1] + '/' + name_split[-1],
    }

In [7]:
def liftover_bed(bed_as_list, chain_abs_path, output_dir='tmp/', keep_files=False, output_name=None):
    '''
    Function for lifting over a list of lists of genomic regions (i.e. list([chrom, start, end])) from
    one genome to another, returning a list of lists. Wraps command line tool LiftoverBED. Output dir
    (default: output_dir=='tmp/') is where files will be written, will be created if does not exist 
    already, should have enough free disk space to write bed files. By default, keep_files==False, so 
    intermediate files will be deleted. Output name is required if keep_files==True.
    
    There is a known issue with the liftover tool where it fails with "Chain mapping error". I've figured
    out this can be avoided if the regions are of at least length 1 (so no identical start and end). A
    check is performed to make sure bed as list does not have identical start and end. 
    '''
    
    which_out = str(subprocess.run(['which', 'LiftoverBED'], stdout=subprocess.PIPE).stdout, encoding=sys.getdefaultencoding())
    if which_out.strip().split('/')[-1] != 'LiftoverBED':
        raise ValueError(
            '''
            LiftoverBED is not installed. Please install from 
            https://genome.sph.umich.edu/wiki/LiftOver, and add to shell PATH
            as `LiftoverBED`.
            '''
        )
    
    for i in bed_as_list:
        if i[1] == i[2]:
            raise ValueError('Known error of liftover tool. Avoid identical start and end loci.')
    
    if output_dir[-1] != '/':
        output_dir += '/'

    if keep_files == False:
        if type(output_name) != type(None):
            print('keep_files == False so output_name is ignored')
        rand_id = str(np.random.randint(low=1000, high=9999))
        output_name = rand_id + '_tmp'
    else:
        if type(output_name) != str:
            raise ValueError('You must set output_name if keep_files == True')
        elif output_name[-4:] == '.bed':
            output_name = output_name[:-4]

    try:
        os.mkdir(output_dir)
        if keep_files == False:
            delete_dir = True
        else:
            delete_dir = False
    except FileExistsError:
        delete_dir = False
    bed_as_list_name = output_name + '_pre_liftover'
    unlifted_name = output_name + '_unlifted'

    with open(output_dir + bed_as_list_name + '.bed', 'w') as file:
        for i in bed_as_list:
            file.write('\t'.join(i) + '\n')

    if os.path.isfile(output_dir + output_name + '.bed'):
        ans = input('Warning – output file %s exists. File will be overwritten. Continue anyway? (y/n)' % (output_dir + output_name + '.bed'))
        if ans == 'y':
            proceed = True
        elif ans == 'n':
            proceed = False
        else:
            raise ValueError('Answer with lowercase y or n')
    else:
        proceed = True

    if proceed:
        completed_process = subprocess.run(['LiftoverBED', 
                                            bed_as_list_name + '.bed', 
                                            chain_abs_path, 
                                            output_name + '.bed',  
                                            unlifted_name + '.bed'], cwd=output_dir)

        with open(output_dir + output_name + '.bed', 'r') as file:
            out_bed_as_list = [i.strip().split('\t') for i in file.readlines()]
        return out_bed_as_list
    else:
        if keep_files == False:
            for fname in [bed_as_list_name, output_name, unlifted_name]:
                os.remove(output_dir + fname + '.bed')
        if delete_dir == True:
            os.rmdir(output_dir)
        return

# Load Data

## VCF

In [ ]:
vcf = load_vcf(prefix_eqtl + 'mateqtl/vcfs/all_chrs_merged_remove_missing_MAF_0.1.recode.vcf.gz')

In [36]:
vcf['gts'].head()

,219,237,1221,1891,DON6,IGTB141,IGTB143,IGTB195,IGTB256,IGTB469,...,IGTB178,3:IGTB469,3:IGTB498,3:IGTB508,IGTB67,IGTB811,IGTB906,IGTB950,3:IGTB986,IGTB993
ID,,,,,,,,,,,,,,,,,,,,,
1:693731:A:G,0|0,0|0,0|1,0|0,0|1,0|0,0|0,0|0,0|0,0|0,...,1|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|1
1:706368:A:G,0|0,0|0,0|0,1|1,1|0,0|0,0|0,1|1,0|1,1|0,...,0|1,0|1,0|0,1|0,0|1,1|1,1|1,0|0,0|0,0|0
1:729679:C:G,1|1,1|1,1|0,1|1,1|0,1|1,0|1,1|1,0|1,1|0,...,0|1,0|1,0|1,1|0,1|1,1|1,1|1,1|1,1|1,1|0
1:731718:T:C,0|0,0|0,0|1,0|0,0|1,0|0,0|0,0|0,0|0,0|0,...,1|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|1
1:734349:T:C,0|0,0|0,0|1,0|0,0|1,0|0,0|0,0|0,0|0,0|0,...,1|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|1


## CSVs

### `demo_ids`

In [53]:
demo_ids = pd.read_csv(prefix_prod + 'vals/demo_ids.csv', index_col=0)

### `fastgxe`

In [114]:
fnames = [fname for fname in os.listdir(prefix_eqtl + 'fastgxe/Control/mateqtl/') if 'heterogeneous' in fname]
cts = [fname.split('_', 1)[1].split('.')[0] for fname in fnames]

In [115]:
fastgxe = pd.concat([pd.read_csv(prefix_eqtl + 'fastgxe/Control/mateqtl/' + fname, sep='\t') for fname in fnames], axis=0, keys=cts)
fastgxe = fastgxe.reset_index(level=0).rename(columns={'level_0': 'cell_type'})

### `mateqtl`

In [54]:
mateqtl_df = pd.read_csv(prefix_eqtl + 'mateqtl/vals/all_results_w_BH_correction.tsv', sep='\t', index_col=0, header=0).sort_values(by='BH')

Due to the `str.strip()` issue (see `/int/codec/production.run/aggr.combat/create.concat.4.ipynb`), I have to remap all of the gene names. When/if matrix eqtl is run on the new cell groupings, we can get rid of this.

#### `str.strip()` mapper

In [55]:
with open(prefix_prod + 'mrna/pkls/mapper.pkl', 'rb') as file:
    gene_strip_mapper = pkl.load(file)

In [56]:
mateqtl_df['gene'] = mateqtl_df['gene'].map(gene_strip_mapper) # str.strip() issue

In [57]:
mateqtl_df['snps:gene'] = mateqtl_df.index.str[:] + ':' + mateqtl_df['gene'].str[:]

In [58]:
mateqtl = dict()
for cond in tqdm(mateqtl_df['cond'].unique()):
    mateqtl[cond] = dict()
    for ct in mateqtl_df[mateqtl_df['cond'] == cond]['cell_type'].unique():
        mateqtl[cond][ct] = mateqtl_df[(mateqtl_df['cond'] == cond) & (mateqtl_df['cell_type'] == ct)].copy()

  0%|          | 0/6 [00:00<?, ?it/s]

## Pseudobulks

In [59]:
pbulk_dir = prefix_eqtl + 'mateqtl/pbulks/'
pbulk_fnames = ! ls $pbulk_dir

In [60]:
pbulk_fnames_split = [i.split('.')[0] for i in pbulk_fnames if i.endswith('.expr')]
pbulk_combos = [tuple(i.split('_', 1)) for i in pbulk_fnames_split]

In [61]:
pbulks = dict()
for cond in np.unique([i[0] for i in pbulk_combos]):
    pbulks[cond] = dict()

In [62]:
for comb in tqdm(pbulk_combos):
    pbulks[comb[0]][comb[1]] = pd.read_csv(pbulk_dir + '_'.join(comb) + '.expr', sep='\t', index_col=0)
    pbulks[comb[0]][comb[1]].index = pbulks[comb[0]][comb[1]].index.map(gene_strip_mapper)

  0%|          | 0/36 [00:00<?, ?it/s]

# Adjust Inputs

## VCF

Make ASB names consistent:

In [57]:
vcf['gts'].columns = ['ASB' + i[-4:] if i.startswith('AS') else i for i in vcf['gts'].columns]

Subset to common names:

In [58]:
common_names = np.intersect1d(vcf['gts'].columns.values, demo_ids.index.values)

In [59]:
vcf['gts'] = vcf['gts'][common_names].copy()
demo_ids = demo_ids.loc[common_names].copy()

Make a version where just the chromosome/position is the index. This will make multiallelic sites ambiguous, but so are the eQTL results.

In [60]:
vcf['gts_snp'] = vcf['gts'].copy()
vcf['gts_snp'].index = vcf['gts_snp'].index.str.split(':').str[:2].str.join(':')

Make a version of both with just integers instead of actual genotypes. In order to do this, I need to map the genotypes.

There is one problematic genotype that I'm just going to drop.

In [62]:
vcf['gts'].index[vcf['gts'].index.str.contains(',')]

Index(['9:14791923:T:A,C'], dtype='object', name='ID')

In [63]:
vcf['gts'].drop(index='9:14791923:T:A,C', inplace=True)

In [44]:
quantize_geno_dict =     {
        '0|0': 0,
        '0|1': 1,
        '1|0': 1,
        '1|1': 2
    }

In [ ]:
vcf['gts_quant'] = vcf['gts'].replace(quantize_geno_dict).astype(np.int8).copy()
vcf['gts_snp_quant'] = vcf['gts_snp'].replace(quantize_geno_dict).astype(np.int8).copy()

### Store as Sparse Matrix

In [45]:
vcf_adata = sc.AnnData(X=sparse.csr_matrix(vcf['gts_quant'].values), obs=vcf['gts_quant'].iloc[:, :0], var=vcf['gts_quant'].iloc[:0, :].T)
vcf_adata.obs = vcf_adata.obs.join(vcf['info'])
vcf_adata.uns['file'] = {'path': vcf['path'], 'header_lines_num': vcf['header_lines_num']}
vcf_adata.uns['uids'] = vcf['uids']
vcf_adata.uns['quantize_geno_dict'] = quantize_geno_dict.copy()

## `pbulks`

Map names:

In [66]:
exp_id_subject_mapper = dict(zip(demo_ids['exp_id'], demo_ids.index))

Rename columns in `pbulks`, subset to common_names by removing `nan`s.

In [67]:
for cond in tqdm(pbulks):
    for ct in pbulks[cond]:
        pbulks[cond][ct].columns = pbulks[cond][ct].columns.astype(int)
        pbulks[cond][ct].columns = pbulks[cond][ct].columns.map(exp_id_subject_mapper)
        pbulks[cond][ct] = pbulks[cond][ct].loc[:, ~pbulks[cond][ct].columns.isna()].copy()

  0%|          | 0/6 [00:00<?, ?it/s]

## eQTL Results

### `mateqtl`

In [63]:
mateqtl_df.dropna(inplace=True)
mateqtl_df['abs(beta)'] = np.abs(mateqtl_df['beta'])
mateqtl_df['-log10FDR'] = -np.log10(mateqtl_df['FDR'])
mateqtl_df_C = mateqtl_df[mateqtl_df['cond'] == 'C'].copy()

In [64]:
for cond in mateqtl:
    for ct in mateqtl[cond]:
        mateqtl[cond][ct]['abs(beta)'] = np.abs(mateqtl[cond][ct]['beta'])
        mateqtl[cond][ct]['-log10FDR'] = -np.log10(mateqtl[cond][ct]['FDR'])

In [65]:
for cond in mateqtl:
    for ct in mateqtl[cond]:
        mateqtl[cond][ct]['abs(beta)'] = np.abs(mateqtl[cond][ct]['beta'])
        mateqtl[cond][ct]['-log10FDR'] = -np.log10(mateqtl[cond][ct]['FDR'])

#### Liftover to hg38

In [ ]:
original_len = mateqtl_df.shape[0]

In [19]:
all_mateqtl_bed = [['chr' +i.split(':')[0], i.split(':')[1], 
                    str(int(i.split(':')[1]) + 1),  # this plus 1 was necessary for samtools depth
                    i] for i in mateqtl_df.index.unique()]

In [27]:
all_mateqtl_bed_hg38 = liftover_bed(all_mateqtl_bed, chain_abs_path=prefix_eqtl + 'hg19ToHg38.over.chain', 
                                    output_dir='/tmp/', keep_files=False)

In [29]:
all_mateqtl_lifted_snps = dict([(i[3], i[0].replace('chr', '') + ':' + i[1]) for i in all_mateqtl_bed_hg38])

The following `.dropna()` will reduce the number of total eQTLs by some.

In [39]:
mateqtl_df['hg38_snps'] = mateqtl_df.index.map(all_mateqtl_lifted_snps)
mateqtl_df.dropna(inplace=True)
mateqtl_df['hg19_snps'] = mateqtl_df.index.to_series()

Still retaining 99.8% of all eQTLs:

In [47]:
mateqtl_df.shape[0]/original_len

0.9985239683689956

### `fastgxe`

In [116]:
fastgxe['abs(beta)'] = np.abs(fastgxe['beta'])
fastgxe['-log10FDR'] = -np.log10(fastgxe['FDR'])

#### Liftover to hg38

In [117]:
original_len = fastgxe.shape[0]

In [118]:
fastgxe.index = fastgxe['SNP'].copy()
fastgxe = fastgxe.rename(columns={'SNP': 'hg19_snps'})

In [119]:
all_fastgxe_bed = [['chr' +i.split(':')[0], i.split(':')[1], 
                    str(int(i.split(':')[1]) + 1),  # this plus 1 was necessary for samtools depth
                    i] for i in fastgxe['hg19_snps'].unique()]

In [120]:
all_fastgxe_bed_hg38 = liftover_bed(all_fastgxe_bed, chain_abs_path=prefix_eqtl + 'hg19ToHg38.over.chain', 
                                    output_dir='/tmp/', keep_files=False)

In [121]:
all_fastgxe_lifted_snps = dict([(i[3], i[0].replace('chr', '') + ':' + i[1]) for i in all_fastgxe_bed_hg38])

The following `.dropna()` will reduce the number of total eQTLs by some.

In [122]:
fastgxe['hg38_snps'] = fastgxe.index.map(all_fastgxe_lifted_snps)
fastgxe.dropna(inplace=True)
fastgxe['hg19_snps'] = fastgxe.index.to_series()

Still retaining 99.8% of all eQTLs:

In [123]:
fastgxe.shape[0]/original_len

0.9998012583484569

# Export

In [7]:
# with open(prefix_eqtl + 'mateqtl/pkls/vcf.pkl', 'wb') as file:
#     pkl.dump(vcf, file, protocol=4)

# with open(prefix_eqtl + 'mateqtl/pkls/vcf.pkl', 'rb') as file:
#     vcf = pkl.load(file)

In [93]:
# vcf_adata.write_h5ad(prefix_eqtl + 'mateqtl/h5ads/vcf_adata.h5ad')
vcf_adata = sc.read_h5ad(prefix_eqtl + 'mateqtl/h5ads/vcf_adata.h5ad')

In [7]:
# with open(prefix_eqtl + 'mateqtl/pkls/pbulks.pkl', 'wb') as file:
#     pkl.dump(pbulks, file, protocol=4)

with open(prefix_eqtl + 'mateqtl/pkls/pbulks.pkl', 'rb') as file:
    pbulks = pkl.load(file)

In [51]:
# mateqtl_df.to_pickle(prefix_eqtl + 'mateqtl/pkls/mateqtl_df.pkl')
mateqtl_df = pd.read_pickle(prefix_eqtl + 'mateqtl/pkls/mateqtl_df.pkl')

In [10]:
# with open(prefix_eqtl + 'mateqtl/pkls/mateqtl.pkl', 'wb') as file:
#     pkl.dump(mateqtl, file, protocol=4)

with open(prefix_eqtl + 'mateqtl/pkls/mateqtl.pkl', 'rb') as file:
    mateqtl = pkl.load(file)

In [ ]:
# fastgxe.to_pickle(prefix_eqtl + 'mateqtl/pkls/fastgxe.pkl')
fastgxe = pd.read_pickle(prefix_eqtl + 'mateqtl/pkls/fastgxe.pkl')